# Benchmark Comparison

This notebook demonstrates how to run benchmarks using the command line interface and visualize the results. It additionally compares QuASAr's native backends with external simulators from Qiskit Aer and MQT DD.


In [ ]:
import json
import sys
import subprocess
import tempfile
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()


In [ ]:
tmp = Path(tempfile.gettempdir())/"quasar_cli_example"
subprocess.run([sys.executable, "../benchmark_cli.py", "--circuit", "ghz", "--qubits", "2:5", "--repetitions", "2", "--output", str(tmp)], check=True)

cli_df = pd.read_csv(tmp.with_suffix(".csv"))

cli_df


In [ ]:
sns.lineplot(data=cli_df, x="qubits", y="avg_time", marker="o")
plt.title("GHZ circuit benchmark")
plt.xlabel("Qubits")
plt.ylabel("Average time (s)")
plt.show()

In [ ]:
nb_root = Path('..' ).resolve()
sys.path.append(str(nb_root))
sys.path.append(str(nb_root.parent))

from runner import BenchmarkRunner
from backends import (
    StatevectorAdapter,
    StimAdapter,
    MPSAdapter,
    DecisionDiagramAdapter,
)
import circuits as circuit_lib
from quasar import SimulationEngine
from quasar.cost import Backend

backends = [StatevectorAdapter()]
runner = BenchmarkRunner()
engine = SimulationEngine()

backend_map = {
    'statevector': Backend.STATEVECTOR,
    'stim': Backend.TABLEAU,
    'mps': Backend.MPS,
    'mqt_dd': Backend.DECISION_DIAGRAM,
}

circuits = {
    'ghz': circuit_lib.ghz_circuit,
    'qft': circuit_lib.qft_circuit,
}

qubit_sizes = list(range(2, 6))

for name, build in circuits.items():
    for n in qubit_sizes:
        try:
            circ = build(n)
        except Exception:
            continue
        for b in backends:
            try:
                rec = runner.run_multiple(circ, b, return_state=False, repetitions=3)
                rec['circuit'] = name
                rec['qubits'] = n
                rec['backend'] = b.name
            except Exception:
                pass
            try:
                rec = runner.run_quasar_multiple(
                    circ,
                    engine,
                    backend=backend_map.get(b.name),
                    repetitions=3,
                )
                rec['circuit'] = name
                rec['qubits'] = n
            except Exception:
                pass


df = pd.DataFrame(runner.results)
df_long = df.melt(
    id_vars=['circuit', 'framework', 'backend', 'qubits'],
    value_vars=['prepare_time_mean', 'run_time_mean'],
    var_name='phase',
    value_name='time',
)
g = sns.relplot(
    data=df_long,
    x='qubits',
    y='time',
    hue='framework',
    style='phase',
    col='circuit',
    col_wrap=3,
    kind='line',
)
g.set(yscale='log')
plt.show()
df[['circuit', 'framework', 'backend', 'qubits', 'prepare_time_mean', 'run_time_mean', 'total_time_mean']]


These results separate the **preparation** time from the actual **simulation** run time. Preparation typically includes converting circuits or planning steps, whereas simulation measures the time spent executing the circuit. The table and plot above illustrate how both components contribute to the total runtime.

In [ ]:
import pandas as pd
from benchmarks.stats_utils import stats_table

def add_stats(df, quasar_col='QuASAr', baseline_cols=None, test='ttest', correction='bonferroni'):
    """Compute statistics comparing QuASAr with baselines.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame with per-circuit results. One column must correspond to QuASAr,
        others to baselines.
    quasar_col : str
        Name of the column containing QuASAr results.
    baseline_cols : list[str] | None
        Columns to treat as baselines. Defaults to all columns except quasar_col.
    test : str
        'ttest' or 'wilcoxon'.
    correction : str
        'bonferroni' or 'fdr_bh'.

    Returns
    -------
    pd.DataFrame
        Table with baseline name, statistic, corrected p-value, and effect size.
    """
    if baseline_cols is None:
        baseline_cols = [c for c in df.columns if c != quasar_col]
    baselines = {c: df[c] for c in baseline_cols}
    return stats_table(df[quasar_col], baselines, test=test, correction=correction)

# Example usage after computing results DataFrame named `results_df`:
# stats_df = add_stats(results_df)
# stats_df


In [ ]:
# Record parameters and results
import json, pathlib
try:
    import ipynbname
    nb_name = ipynbname.path().stem
except Exception:  # pragma: no cover
    nb_name = 'notebook'

# Collect simple parameters from globals
_params = {
    k: v for k, v in globals().items()
    if not k.startswith('_') and isinstance(v, (int, float, str, bool, list, dict, tuple))
}
pathlib.Path('../results').mkdir(exist_ok=True)
with open(f'../results/{nb_name}_params.json', 'w') as f:
    json.dump(_params, f, indent=2, default=str)
if 'results' in globals():
    try:
        with open(f'../results/{nb_name}_results.json', 'w') as f:
            json.dump(results, f, indent=2)
    except TypeError:
        pass
print(json.dumps(_params, indent=2, default=str))
